In [1]:
import os
import cv2 as cv
import numpy as np
from PIL import Image
import torch
from torch import nn
from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [3]:
def load_images_from_folder(folder):
    images = []
    for file in os.listdir(folder):
        img = cv.imread(os.path.join(folder,file))
        if(img is not None):
            images.append(img)
    return images

In [4]:
def resize_and_flatten(img, target_size=(64,64)):
    resized_img = cv.resize(img, target_size)
    flattened_img = resized_img.reshape(target_size[0] * target_size[1] * 3)
    return flattened_img

In [5]:
cat_data = []
for img in load_images_from_folder("../data/cat"):
    cat_data.append(resize_and_flatten(img))
dog_data = []
for img in load_images_from_folder("../data/dog"):
    dog_data.append(resize_and_flatten(img))

In [6]:
X = []
y = []

for entity in cat_data:
    X.append(entity)
    y.append(1)

for entity in dog_data:
    X.append(entity)
    y.append(0)

np.array(X).shape, np.array(y).shape

((160, 12288), (160,))

In [7]:
class MyNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(64*64*3, 16*16),
            nn.ReLU(),
            nn.Linear(16*16, 16*16),
            nn.ReLU(),
            nn.Linear(16*16, 2),
            nn.Sigmoid()
        )
    def forward(self, x, **kwargs):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [8]:
net = NeuralNetClassifier(
    MyNN,
    max_epochs=10,
    lr=0.1,
    iterator_train__shuffle=True,
)


In [9]:
params = {    
    'lr': [0.001, 0.01, 0.0001],
    'max_epochs': [1,2,5, 10, 20, 25],
}
gs = GridSearchCV(net, params, cv=2)
gs.fit(np.array(X, dtype='float32'), np.array(y))
pd.DataFrame(gs.cv_results_)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1       15.9423       0.3750       15.9424  0.0092
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        3.7194       0.5000        7.9712  0.0065
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        7.9765       0.5000        7.9712  0.0081
      2        7.9712       0.5000        7.9712  0.0077
      3        7.9712       0.5000        7.9712  0.0086
      4        7.9712       0.5000        7.9712  0.0086
      5        7.9712       0.5000        7.9712  0.0090
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        7.9717       0.5000        7.9712  0.0071
      2        7.9712       0.5000        7.9712  0.0075
      3        7.9712       0.5

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_lr,param_max_epochs,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
0,0.019761,0.002195,0.004434,0.000649,0.001,1,"{'lr': 0.001, 'max_epochs': 1}",0.5125,0.5,0.50625,0.00625,1
1,0.053733,0.002569,0.004383,0.000338,0.001,5,"{'lr': 0.001, 'max_epochs': 5}",0.5000,0.5,0.50000,0.00000,2
2,0.096866,0.000423,0.004085,0.000240,0.001,10,"{'lr': 0.001, 'max_epochs': 10}",0.5000,0.5,0.50000,0.00000,2
3,0.187598,0.000326,0.003877,0.000116,0.001,20,"{'lr': 0.001, 'max_epochs': 20}",0.4750,0.5,0.48750,0.01250,15
4,0.230991,0.007514,0.004178,0.000526,0.001,25,"{'lr': 0.001, 'max_epochs': 25}",0.5000,0.5,0.50000,0.00000,2
5,0.019057,0.000168,0.004238,0.000392,0.01,1,"{'lr': 0.01, 'max_epochs': 1}",0.5000,0.5,0.50000,0.00000,2
6,0.060081,0.002129,0.004253,0.000473,0.01,5,"{'lr': 0.01, 'max_epochs': 5}",0.5000,0.5,0.50000,0.00000,2
7,0.106375,0.002573,0.003936,0.000013,0.01,10,"{'lr': 0.01, 'max_epochs': 10}",0.5000,0.5,0.50000,0.00000,2
8,0.201362,0.003212,0.004985,0.000804,0.01,20,"{'lr': 0.01, 'max_epochs': 20}",0.5000,0.5,0.50000,0.00000,2
9,0.237699,0.003112,0.004231,0.000121,0.01,25,"{'lr': 0.01, 'max_epochs': 25}",0.5000,0.5,0.50000,0.00000,2
